In [ ]:
#Montar Colab en Drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install datasets


     |████████████████████████████████| 7.6 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 3.9 MB/s eta 0:00:01


In [1]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding, pipeline
from datasets import load_metric, Dataset
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
#snips_file = open('/content/drive/MyDrive/snips.train.txt', 'rb')
snips_file = open('./snips.train.txt', 'rb')

snips_rows = snips_file.readlines()

snips_rows[:20]

[b'listen O\r\n',
 b'to O\r\n',
 b'westbam B-artist\r\n',
 b'alumb O\r\n',
 b'allergic B-album\r\n',
 b'on O\r\n',
 b'google B-service\r\n',
 b'music I-service\r\n',
 b'PlayMusic\r\n',
 b'\r\n',
 b'add O\r\n',
 b'step B-entity_name\r\n',
 b'to I-entity_name\r\n',
 b'me I-entity_name\r\n',
 b'to O\r\n',
 b'the O\r\n',
 b'50 B-playlist\r\n',
 b'cl\xc3\xa1sicos I-playlist\r\n',
 b'playlist O\r\n',
 b'AddToPlaylist\r\n']

In [3]:
utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []

utterance, tokenized_utterance, label_for_utterances = '', [], []
for snip_row in snips_rows:
    if len(snip_row) == 2:  # saltarse las filas sin datos
        continue
    if ' ' not in snip_row.decode():  # we've hit a sequence label
        sequence_labels.append(snip_row.decode().strip())
        utterances.append(utterance.strip())
        tokenized_utterances.append(tokenized_utterance)
        labels_for_tokens.append(label_for_utterances)
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.decode().split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)

In [4]:
len(labels_for_tokens), len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [5]:
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels


['GetWeather',
 'RateBook',
 'SearchCreativeWork',
 'BookRestaurant',
 'SearchScreeningEvent',
 'PlayMusic',
 'AddToPlaylist']

In [6]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

print(f'There are {len(unique_sequence_labels)} unique sequence labels')

There are 7 unique sequence labels


In [7]:
from functools import reduce

unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))
labels_for_tokens = [[unique_token_labels.index(_) for _ in l] for l in labels_for_tokens]

print(f'There are {len(unique_token_labels)} unique token labels')

There are 72 unique token labels


In [8]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([unique_token_labels[l] for l in labels_for_tokens[0]])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[46, 46, 8, 46, 35, 46, 53, 19]
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
5
PlayMusic


In [9]:
snips_dataset = Dataset.from_dict(
    dict(
        utterance=utterances,
        label=sequence_labels,
        tokens=tokenized_utterances,
        token_labels=labels_for_tokens
    )
)
snips_dataset = snips_dataset.train_test_split(test_size=0.2)

In [10]:
snips_dataset['train'][0]

{'utterance': 'where is the closest cinema playing good night good morning',
 'label': 4,
 'tokens': ['where',
  'is',
  'the',
  'closest',
  'cinema',
  'playing',
  'good',
  'night',
  'good',
  'morning'],
 'token_labels': [46, 46, 46, 67, 30, 46, 44, 57, 57, 57]}

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [12]:
# función simple para tokenizar por lotes expresiones con truncamiento
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True)

In [13]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10467 [00:00<?, ? examples/s]

Map:   0%|          | 0/2617 [00:00<?, ? examples/s]

In [14]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'where is the closest cinema playing good night good morning',
 'label': 4,
 'tokens': ['where',
  'is',
  'the',
  'closest',
  'cinema',
  'playing',
  'good',
  'night',
  'good',
  'morning'],
 'token_labels': [46, 46, 46, 67, 30, 46, 44, 57, 57, 57],
 'input_ids': [101,
  2073,
  2003,
  1996,
  7541,
  5988,
  2652,
  2204,
  2305,
  2204,
  2851,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
# DataCollatorWithPadding crea un lote de datos. También rellena dinámicamente el texto con la
# longitud del elemento más largo en el lote, haciéndolos todos de la misma longitud.
# Es posible rellenar el texto en la función tokenizadora con padding=True, pero el relleno dinámico es más eficiente.

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
# Data Collator rellenará los datos para que todos los ejemplos tengan la misma longitud de entrada.
# La máscara de atención es cómo ignoramos los puntajes de atención para los tokens de relleno


In [17]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(unique_sequence_labels),
)

# establecer un índice -> diccionario de etiquetas
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
sequence_clf_model.config.id2label[0]

'GetWeather'

In [25]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 81 kB 4.7 MB/s eta 0:00:01


In [26]:
from evaluate import load

In [27]:
# metric = load_metric("accuracy")  #Deprecated
metric = load("accuracy")

def compute_metrics(eval_pred):  # método personalizado para tomar logits y calcular la precisión del conjunto de evaluación
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [28]:
import os

# Creamos un directorio en el que guardar las pruebas y evaluaciones de nuestro entrenamiento

#os.makedirs("/content/drive/MyDrive/snips_clf/results", exist_ok=True)
os.makedirs("./snips_clf/results", exist_ok=True)

In [29]:
!pip install accelerate -U
!pip install transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: no matches found: transformers[torch]


In [48]:
epochs = 2

# Este bloque es el fundamental y que siempre se usa en todos los modelos que usan arquitectura transformers (BERT,GPT,etc)

# Definimos los parametros basicos del entrenamiento
training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,  # Ciclos de entrenamiento con los datos completos
    per_device_train_batch_size=32, #Numeros de secuencias que se cargaran por lote y que se procesaran simultaneamente
    per_device_eval_batch_size=32,
    load_best_model_at_end=True, #Evita el "sobreentrenamiento" al carga el checkpoint con evaluacion de perdida mas baja
    report_to="all",
    
    # algunos parámetros de aprendizaje profundo que el Trainer puede tomar
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,  # número de pasos de calentamiento para el programador de tasa de aprendizaje
    weight_decay = 0.05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    eval_steps=50,
    save_strategy='epoch'
    
)

# Define el trainer:

trainer = Trainer(
    model=sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices


In [49]:
# Get initial metrics
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 0.052907392382621765,
 'eval_accuracy': 0.9896828429499427,
 'eval_runtime': 3.3252,
 'eval_samples_per_second': 787.024,
 'eval_steps_per_second': 24.66}

In [50]:
# 'eval_accuracy': 0.1841803591899121,  Antes del entrenamiento la exactitud en las predicciones es del 18%, como hay 5 etiqutas, es azar

In [51]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66,958,855


Epoch,Training Loss,Validation Loss,Accuracy
1,0.007900,0.046111,0.989683
2,0.002200,0.061571,0.985480


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_clf/results/checkpoint-328
Configuration saved in ./snips_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_clf/results/checkpoint-328/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Sa

TrainOutput(global_step=656, training_loss=0.031466283825532786, metrics={'train_runtime': 101.4283, 'train_samples_per_second': 206.392, 'train_steps_per_second': 6.468, 'total_flos': 116090859411654.0, 'train_loss': 0.031466283825532786, 'epoch': 2.0})

In [52]:
# La exactitud en las predicciones despues del primer epoch pasa a ser del 97,89%, y en el segundo sube un poco

In [53]:
 # Ahora evaluamos con las sentencias de evaluacion, 

In [54]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 0.04611092060804367,
 'eval_accuracy': 0.9896828429499427,
 'eval_runtime': 3.3396,
 'eval_samples_per_second': 783.635,
 'eval_steps_per_second': 24.554,
 'epoch': 2.0}

In [55]:
import torch

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", sequence_clf_model, tokenizer=tokenizer,device=device)
pipe('Please add Here We Go by Dispatch to my road trip playlist')

[{'label': 'AddToPlaylist', 'score': 0.995516836643219}]

In [57]:
trainer.save_model()

Saving model checkpoint to ./snips_clf/results
Configuration saved in ./snips_clf/results/config.json
Model weights saved in ./snips_clf/results/pytorch_model.bin


In [58]:
pipe = pipeline("text-classification", "./snips_clf/results", tokenizer=tokenizer,device=device)
pipe('Please add Here We Go by Dispatch to my road trip playlist')

loading configuration file ./snips_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "GetWeather",
    "1": "RateBook",
    "2": "SearchCreativeWork",
    "3": "BookRestaurant",
    "4": "SearchScreeningEvent",
    "5": "PlayMusic",
    "6": "AddToPlaylist"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transfor

[{'label': 'AddToPlaylist', 'score': 0.995516836643219}]

In [59]:
frozen_sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(unique_sequence_labels),
)

loading configuration file config.json from cache at /home/ortiz/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.33.1",
  "vocab_size": 30522
}

lo

In [60]:
for param in frozen_sequence_clf_model.distilbert.parameters():
    param.requires_grad = False #Congelamos todos los gradientes

In [61]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    # some deep learning parameters that the Trainer is able to take in
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,  # number of warmup steps for learning rate scheduler,
    weight_decay = 0.05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    eval_steps=50,
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=frozen_sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 1.9606684446334839,
 'eval_accuracy': 0.13335880779518533,
 'eval_runtime': 128.9482,
 'eval_samples_per_second': 20.295,
 'eval_steps_per_second': 0.636}

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 595,975


Epoch,Training Loss,Validation Loss,Accuracy
1,1.930000,1.891287,0.422239
2,1.819300,1.685474,0.835690


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


Saving model checkpoint to /content/drive/MyDrive/snips_clf/results/checkpoint-328
Configuration saved in /content/drive/MyDrive/snips_clf/results/checkpoint-328/config.json
Model weights saved in /content/drive/MyDrive/snips_clf/results/checkpoint-328/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/snips_clf/results/checkpoint-656
Configuration saved in /content/drive/MyDrive/snips_clf/results/checkpoint-656/config.json
Model weights saved in /content/drive/MyDrive/snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on hugg

TrainOutput(global_step=656, training_loss=1.8748308138512984, metrics={'train_runtime': 1187.4439, 'train_samples_per_second': 17.629, 'train_steps_per_second': 0.552, 'total_flos': 130773540660864.0, 'train_loss': 1.8748308138512984, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_labels, tokens. If utterance, token_labels, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 1.6854736804962158,
 'eval_accuracy': 0.8356897210546427,
 'eval_runtime': 121.4216,
 'eval_samples_per_second': 21.553,
 'eval_steps_per_second': 0.675,
 'epoch': 2.0}